# Apple stock forecasting 

In [2]:
# general
import pandas as pd
import numpy as np 
from datetime import datetime, date, timedelta

# stock data/info
import yfinance as yf

# news sentiment
from GoogleNews import GoogleNews
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# ML 
import tensorflow as tf


# set todays date
today = date.today()

## Get stock info

In [87]:
#stocks = ['BAC', 'AAPL', 'AMZN', 'LSXMA']
#LSTM
appl = yf.Ticker("AAPL")

In [88]:
historical_data = appl.history(period="max").reset_index()

In [89]:
recc_data = appl.recommendations.reset_index()

In [91]:
historical_data = historical_data[historical_data['Date'] > datetime(2012, 2, 8)]

In [92]:
historical_data['Date'] = historical_data.Date.astype('str')

In [93]:
# change date format to be consistent 
recc_data['Date'] = recc_data["Date"].dt.strftime("%Y-%m-%d")
recc_data.head()

,Date,Firm,To Grade,From Grade,Action
0,2012-02-09,Canaccord Genuity,Buy,,main
1,2012-03-08,FBN Securities,Outperform,,main
2,2012-03-13,Jefferies,Buy,,main
3,2012-03-14,Morgan Stanley,Overweight,,main
4,2012-03-14,Canaccord Genuity,Buy,,main


In [94]:
#leave out firm for now 
recc_data = recc_data[['Date','To Grade', 'From Grade', 'Action']]

In [95]:
df = historical_data.merge(recc_data, on='Date', how='left')

In [96]:
for index, row in df.iterrows(): row['Date'] = datetime.strptime(row['Date'], '%Y-%m-%d')

In [97]:
df.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,To Grade,From Grade,Action
0,2012-02-09,14.764531,15.255597,14.758388,15.145652,884214800,0.0,0.0,Buy,,main
1,2012-02-10,15.077783,15.282316,15.003769,15.153331,631302000,0.0,0.0,NaN,NaN,NaN
2,2012-02-13,15.340975,15.473032,15.266040,15.435258,517216000,0.0,0.0,NaN,NaN,NaN
3,2012-02-14,15.498522,15.649004,15.416832,15.645934,460398400,0.0,0.0,NaN,NaN,NaN
4,2012-02-15,15.793348,16.162798,15.259901,15.283856,1506120000,0.0,0.0,NaN,NaN,NaN


In [99]:
grade_dict = {'': 0, 'Buy': 1, 'Outperform': 2, 'Overweight': 3, 'Neutral': 4, 'Market Perform': 5, 'Hold': 6, 'Equal-Weight': 7, 'Equal-weight': 7, 'Strong Buy': 8,
                  'Long-Term Buy': 9, 'Long-term Buy': 9,'Sell': 10, 'Sector Weight': 11, 'Equal-weight': 12, 'Sector Perform': 13, 'Reduce': 14, 'Accumulate': 15, 
                  'Perform': 16, 'Positive':17, 'Market Outperform': 18, 'Sector Outperform': 19, 'Underperform': 20, 'Negative': 21, 'Underweight': 22, '': -1}


action_dict = {'main': 0, 'down': 1, 'up': 2, 'init': 3, 'reit': 4, '': -1}

df.replace({'To Grade': grade_dict}, inplace=True)
df.replace({'From Grade': grade_dict}, inplace=True)
df.replace({'Action': action_dict}, inplace=True)

In [101]:
df['To Grade'] = df['To Grade'].fillna(-1)
df['From Grade'] = df['From Grade'].fillna(-1)
df['Action'] = df['Action'].fillna(-1)

In [102]:
df.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,To Grade,From Grade,Action
0,2012-02-09,14.764531,15.255597,14.758388,15.145652,884214800,0.0,0.0,1.0,-1.0,0.0
1,2012-02-10,15.077783,15.282316,15.003769,15.153331,631302000,0.0,0.0,-1.0,-1.0,-1.0
2,2012-02-13,15.340975,15.473032,15.266040,15.435258,517216000,0.0,0.0,-1.0,-1.0,-1.0
3,2012-02-14,15.498522,15.649004,15.416832,15.645934,460398400,0.0,0.0,-1.0,-1.0,-1.0
4,2012-02-15,15.793348,16.162798,15.259901,15.283856,1506120000,0.0,0.0,-1.0,-1.0,-1.0


## News collection

In [181]:
#reload collected data
news_df = pd.read_excel('gnews.xlsx')
news_df['link'] = ""
news_df['img'] = ""
news_df['media'] = ""
#news_df['datetime'] = ""

In [182]:
#work around for api only returning 10 results in a date range

#too many requests so collect over time 
#write back in from csv file

# add counter to track how many requests a loop can take 

googlenews = GoogleNews()

def daterange(start_date, end_date):
    for i in range(int((end_date - start_date).days)):
        yield start_date + timedelta(i)

start_date = date(2013, 12, 14)
end_date = today
#count = 0

for single_date in daterange(start_date, end_date):
    googlenews.set_time_range(str(single_date.strftime('%m/%d/%Y')), str(single_date.strftime('%m/%d/%Y')))
    googlenews.search('Apple')
    temp = pd.DataFrame(googlenews.results())
    news_df = news_df.append(temp, ignore_index=True)
    #count += 1
    

HTTP Error 429: Too Many Requests
HTTP Error 429: Too Many Requests


KeyboardInterrupt: 

In [183]:
news_df.tail()

,Unnamed: 0,title,date,desc,link,img,media,datetime
626639,NaN,Beatles bootlegs headed to iTunes next week,"Dec. 13, 2013",Rights owner Apple Corps. and Apple Inc. were ...,https://www.theverge.com/2013/12/13/5209002/be...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",The Verge,NaT
626640,NaN,Kim Kardashian's stepfather Bruce Jenner consi...,"Dec. 13, 2013",Kim Kardashian's stepfather Bruce Jenner consi...,https://www.mirror.co.uk/3am/celebrity-news/br...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",The Mirror,NaT
626641,NaN,A New OS X 11 Concept Visualizes Apple's iOS O...,"Dec. 13, 2013",Apple has pushed a number of iOS features onto...,https://appadvice.com/appnn/2013/12/a-new-os-x...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",AppAdvice,NaT
626642,NaN,"Fruity Cocktail Syrups, Mail-order Artisanal B...","Dec. 13, 2013","Consider Quince & Apple, the Madison, Wis.-bas...",https://www.wsj.com/articles/SB100014240527023...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",The Wall Street Journal,NaT
626643,NaN,Motorola's chief executive: 'our closest compe...,"Dec. 13, 2013",Motorola's chief executive: 'our closest compe...,https://www.theguardian.com/technology/2013/de...,"data:image/gif;base64,R0lGODlhAQABAIAAAP//////...",The Guardian,NaT


In [184]:
len(news_df)

626644

In [185]:
news_df2 = news_df[['title', 'date',   'desc']]

In [ ]:
news_df2.to_excel('gnews.xlsx')

## News sentiment analysis

use vader rather than training own model - dont have labelled data anyways

In [72]:
sa = pd.read_excel('news.xlsx')

In [73]:
len(sa)

1048575

In [74]:
analyzer = SentimentIntensityAnalyzer()

def vader_sentiment_result(sent):
    scores = analyzer.polarity_scores(sent)
    
    if scores["neg"] > scores["pos"]:
        return 0

    return 1

sa["sentiment"] = sa["title"].apply(lambda x: vader_sentiment_result(x))


In [75]:
sa

,Unnamed: 0,title,date,desc,sentiment
0,0,PSA: iPhone alarms not working come New Year's...,"Jan. 1, 2011",We're not exactly sure of the cause of this fa...,0
1,1,"Windows, OS X come to the Google Cr-48 Chrome ...","Jan. 1, 2011",This person has infringed on Apple's rights. T...,1
2,2,Top 10 Jailbreak Apps And Tweaks For 2011,"Jan. 1, 2011","With iOS 4, Apple introduced folders in the sp...",1
3,3,PSA: iPhone alarms not working come New Year's...,"Jan. 1, 2011",We're not exactly sure of the cause of this fa...,0
4,4,"Windows, OS X come to the Google Cr-48 Chrome ...","Jan. 1, 2011",This person has infringed on Apple's rights. T...,1
...,...,...,...,...,...
1048570,1048570,Apple Watch 'Magnetic Wristband' Patent Could ...,"Jan. 14, 2016",The United States Patent and Trademark Office ...,1
1048571,1048571,Apple Music Met With Open Arms (and Lots of Su...,"Jan. 14, 2016",The number of Apple Music subscribers has repo...,1
1048572,1048572,The 2012 Non-Retina MacBook Pro Is Still the B...,"Jan. 14, 2016",Apple's MacBook lineup now features the imposs...,1
1048573,1048573,Bug displays Chrome user's porn hours later on...,"Jan. 14, 2016",Apple has not responded to a request for comme...,1


In [76]:
sa.to_excel('news2_with_sentiment.xlsx')

In [ ]:
# two columns 
# 1. count of good sentiment per date
# 2. count of bad sentiment per date

## Generate final dataframe

In [17]:
news = pd.read_excel('news_with_sentiment.xlsx')
news2 = pd.read_excel('news2_with_sentiment.xlsx')
news3 = pd.read_excel('news3_with_sentiment.xlsx')

In [18]:
news = news.groupby(['date']).sum('sentiment').reset_index()[['date', 'sentiment']]
news2 = news2.groupby(['date']).sum('sentiment').reset_index()[['date', 'sentiment']]
news3 = news3.groupby(['date']).sum('sentiment').reset_index()[['date', 'sentiment']]

In [24]:
news_total = news.append(news2).append(news3)

In [25]:
len(news_total)

4257

In [33]:
# match date format to 
#date_time_str = '18/09/19 01:55:19'
x = 0
for index, row in news_total.iterrows():
    try:
        row['date'] = datetime.strptime(row['date'], '%b. %d, %Y').dt.strftime("%Y-%m-%d")
    except:
        x = 1

news_total.head()

,date,sentiment
0,3 days ago,0
1,5 days ago,28
2,6 days ago,0
3,"Apr. 1, 2011",516
4,"Apr. 1, 2012",828


In [ ]:
# DF

## Model training 

In [ ]:
# train and test model
# set up pipeline to continuously retrain 
# make a webapp 